# LIDC-IDRI extraction debug

In [48]:
DATAPATH = '/home/g/Desktop/git/nodules/cv-bins/'
BIN0PATH = '/scr/nodules/lidc-idri/bin0/'
BIN1PATH = '/scr/nodules/lidc-idri/bin1/'
BIN2PATH = '/scr/nodules/lidc-idri/bin2/'
BIN3PATH = '/scr/nodules/lidc-idri/bin3/'
BIN4PATH = '/scr/nodules/lidc-idri/bin4/'
BIN5PATH = '/scr/nodules/lidc-idri/bin5/'
BIN6PATH = '/scr/nodules/lidc-idri/bin6/'
BIN7PATH = '/scr/nodules/lidc-idri/bin7/'
BIN8PATH = '/scr/nodules/lidc-idri/bin8/'
BIN9PATH = '/scr/nodules/lidc-idri/bin9/'
ANNOTATIONS = '/home/g/Desktop/git/nodules/resources/annotations.csv'

In [104]:
df = pd.read_csv(ANNOTATIONS)
print df[df.seriesuid=='1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968543712594665815359']

                                             seriesuid     coordX     coordY  \
897  1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968... -57.164560  44.511321   
898  1.3.6.1.4.1.14519.5.2.1.6279.6001.323859712968... -80.955332 -74.336800   

         coordZ  diameter_mm  
897  -79.737937     6.614419  
898 -115.057661     5.104385  


In [42]:
import dicom
import os
import math
import time
import xml.etree.ElementTree as et
import skimage.measure as measure
import scipy.ndimage
import numpy as np
import pandas as pd

In [89]:
class Scan:
  def __init__(self):
    self.nodules = []
    self.newNodules = []
    self.groups = []
    self.largeGroups = []
    self.id = None
    
    self.xmlpath = ''
    
    self.sliceList = []
    self.sliceID = {}
    self.locationID = {}
    self.locationDict = {}
    self.locationDictAlt = {}
    
  def combine():
    for group in groups:
      if nodules[group[0]].type == 'S':
        nn = Nodule()
        nn.diameter = 3.
        cx, cy, cz = 0.,0.,0.
        for i in range(len(group)):
          cx += nodules[group[i]].centroid[0]
          cy += nodules[group[i]].centroid[1]
          cz += nodules[group[i]].centroid[2]
        newCentroid = np.array([cx,cy,cz])
        newCentroid /= float(len(group))
        nn.centroid = newCentroid
        nn.tagCount = len(group)
        self.newNodules.append(nn)
      elif nodules[group[0]].type == 'N':
        nn = Nodule()
        nn.diameter = 5.
        cx, cy, cz = 0.,0.,0.
        for i in range(len(group)):
          cx += nodules[group[i]].centroid[0]
          cy += nodules[group[i]].centroid[1]
          cz += nodules[group[i]].centroid[2]
        newCentroid = np.array([cx,cy,cz])
        newCentroid /= float(len(group))
        nn.centroid = newCentroid
        nn.tagCount = len(group)        
        self.newNodules.append(nn)
      elif nodules[group[0]].type == 'L':
        nn = Nodule()
        cx,cy,cz,d = 0.,0.,0.,0.
        for i in range(len(group)):
          cx += nodules[group[i]].centroid[0]
          cy += nodules[group[i]].centroid[1]
          cz += nodules[group[i]].centroid[2]
          d += nodules[group[i]].diameter
          nn.subtlety += nodules[group[i]].subtlety
          nn.internalStructure += nodules[group[i]].internalStructure
          nn.sphericity += nodules[group[i]].shpericity 
          nn.calcification += nodules[group[i]].calcification
          nn.margin += nodules[group[i]].margin
          nn.lobulation += nodules[group[i]].lobulation
          nn.spiculation += nodules[group[i]].spiculation
          nn.texture += nodules[group[i]].texture
          nn.malignancy += nodules[group[i]].malignancy
          nn.rois.append(nodules[group[i]].roi)
        newCentroid = np.array([cx,cy,cz])
        newCentroid /= float(len(group))
        nn.centroid = newCentroid
        nn.tagCount = len(group)
        nn.diameter = d/float(len(group))
        nn.subtlety /= float(len(group))
        nn.intenalStructure /= float(len(group))
        nn.sphericity /= float(len(group))
        nn.calcification /= float(len(group))
        nn.margin /= float(len(group))
        nn.lobulation /= float(len(group))
        nn.spiculation /= float(len(group))
        nn.texture /= float(len(group))
        nn.malignancy /= float(len(group))
        self.newNodules.append(nn)

In [90]:
class Nodule(Scan):
  def __init__(self):
    self.type = ''
    self.spacing = None
    self.no = -1
    
    self.subtlety = 0.
    self.internalStructure = 0.
    self.shpericity = 0.
    self.calcification = 0.
    self.margin = 0.
    self.lobulation = 0.
    self.spiculation = 0.
    self.texture = 0.
    self.malignancy = 0.
    
    #original x and y
    self.ox = 0.
    self.oy = 0. 
    
    self.plane = None
    self.planes = []
    self.roi = {}
    self.rois = []
    
    self.centroid = None
    self.area = None
    self.diameter = None
    
    self.group = -1
    self.grouped = False
    self.groupMembers = []
    self.tagCount = 0
    
    self.AttributeError = False

    
  def addGroup(self,groupNo,group):
    self.group = groupNo
    self.grouped = True
    for i in range(len(group)):
      if group[i] != self.no:
        self.groupMembers.append(group[i])
        
  def addCentroid(self):
    centroid = np.array([self.ox,self.oy,self.plane])
    
  def getProperties(self,spacing):
    tmp = np.zeros((500,500,500)).astype(int)
    xmax,xmin,ymax,ymin,zmax,zmin=-1e5,1e5,-1e5,1e5,-1e5,1e5
    x,y,z=0,0,0
    for key, values in self.roi.iteritems():
      for value in values:
        zmax = max(abs(int(key)),zmax)
        zmin = min(abs(int(key)),zmin)
        xmax = max(abs(int(value[0])),xmax)
        xmin = min(abs(int(value[0])),xmin)
        ymax = max(abs(int(value[1])),ymax)
        ymin = min(abs(int(value[1])),ymin)
        x,y,z = value[0],value[1],key
        tmp[abs(int(x)),abs(int(y)),abs(int(z))]=1  

    props = measure.regionprops(tmp)
    centroid = np.array(props[0].centroid)
    centroid[0] *= int(x)/abs(int(x))
    centroid[1] *= int(y)/abs(int(y))
    centroid[2] *= int(z)/abs(int(z))
    self.centroid = centroid
    
    tmp = tmp[xmin-1:xmax+1,ymin-1:ymax+1,zmin-1:zmax+1]
    tmp = scipy.ndimage.interpolation.zoom(tmp,spacing,mode='nearest')
    tmp[tmp>0] = 1
    if tmp.shape[0]*tmp.shape[1]*tmp.shape[2] == max(tmp.shape):
      self.diameter = 3.
      self.area = max(tmp.shape)
    else:
      props = measure.regionprops(tmp)
      self.diameter = props[0].equivalent_diameter
      self.area = props[0].area

In [91]:
def getDistance(a,b,spacing):
  du = (a[0]-b[0])*spacing[0]
  dv = (a[1]-b[1])*spacing[1]
  dw = (a[2]-b[2])*spacing[2]
  return math.sqrt(du*du+dv*dv+dw*dw)

In [92]:
def cluster(scan):
  print scan.id
  nodules = []
  small = []
  non = []
  gno = 0

  for i, nodule in enumerate(scan.nodules):
    if nodule.type == 'L':
      nodule.no = i
      nodules.append(nodule)
    elif nodule.type == 'S':
      nodule.addCentroid()
      small.append(nodule)
    else:
      nodule.addCentroid()
      non.append(nodule)
    
  for i in range(len(nodules)):
    if not nodules[i].grouped:
      group = []
      group.append(i)
      for j in range(len(nodules)):
        if i != j:
          if not nodules[j].grouped:
            distance = getDistance(nodules[i].centroid,nodules[j].centroid,scan.spacing)
            if distance < (nodules[i].diameter/2.+nodules[j].diameter/2.):
              group.append(j)
      if len(group) < 4:
        for j in range(1,len(group)):
          j = group[j]
          for k in range(len(nodules)):
            if i != k and j != k and i != j:
              if not nodules[k].grouped:
                distance = getDistance(nodules[j].centroid,nodules[k].centroid,scan.spacing)
                if distance < (nodules[j].diameter/2.+nodules[k].diameter/2.):
                  group.append(k)
      if len(group) >= 1:
        group = set(group)
        group = list(group)
        for n in range(len(group)):
          nodules[group[n]].addGroup(gno,group)
        scan.groups.append(group)
        scan.largeGroups.append(group)
        gno += 1

  for i in range(len(small)):
    if not small[i].grouped:
      group = []
      group.append(i)
      for j in range(len(small)):
        if i != j:
          if not small[j].grouped:
            distance = getDistance(small[i].centroid,small[j].centroid,scan.spacing)
            if distance < (small[i].diameter/2.+small[j].diameter/2.):
              group.append(j)
      if len(group) < 4:
        for j in range(1,len(group)):
          j = group[j]
          for k in range(len(small)):
            if i != k and j != k and i != j:
              if not small[k].grouped:
                distance = getDistance(small[j].centroid,small[k].centroid,scan.spacing)
                if distance < (small[j].diameter/2.+small[k].diameter/2.):
                  group.append(k)
      if len(group) >= 1:
        group = set(group)
        group = list(group)
        for n in range(len(group)):
          small[group[n]].addGroup(gno,group)
        scan.groups.append(group)
        gno += 1
        
  for i in range(len(non)):
    if not non[i].grouped:
      group = []
      group.append(i)
      for j in range(len(non)):
        if i != j:
          if not non[j].grouped:
            distance = getDistance(non[i].centroid,non[j].centroid,scan.spacing)
            if distance < (non[i].diameter/2.+non[j].diameter/2.):
              group.append(j)
      if len(group) < 4:
        for j in range(1,len(group)):
          j = group[j]
          for k in range(len(non)):
            if i != k and j != k and i != j:
              if not non[k].grouped:
                distance = getDistance(non[j].centroid,non[k].centroid,scan.spacing)
                if distance < (non[j].diameter/2.+non[k].diameter/2.):
                  group.append(k)
      if len(group) >= 1:
        group = set(group)
        group = list(group)
        for n in range(len(group)):
          non[group[n]].addGroup(gno,group)
        scan.groups.append(group)
        gno += 1

In [93]:
#scans = getFilesList(BIN0PATH)
#readXML(scans[5])
#cluster(scans[5])

In [126]:
def collectXML(path,outpath):
  for parent, subdir, files in os.walk(path):
    for filename in files:
      if ".xml" in filename.lower():
        xmlpath = os.path.join(parent, filename)
        xml = et.parse(xmlpath)
        header = xml.find('./{http://www.nih.gov}ResponseHeader')
        if header is not None:
          seriesuid = header.find('./{http://www.nih.gov}SeriesInstanceUid')
          newfilename = seriesuid.text + '.xml'
          newpath = os.path.join(outpath, newfilename)
          os.rename(xmlpath,newpath)
path = '/scr/nodules/LIDC-IDRI/'
newpath = '/scr/nodules/xml/'
collectXML(path,newpath)

Note: use same bins when processing

In [111]:
def getFilesList(path):
  scans = []
  df = pd.read_csv(ANNOTATIONS)
  for parent, subdir, files in os.walk(path):
    if (len(files)) > 64:
      newscan = Scan()
      locationList = []
      locationListAlt = []    
      for filename in files:
        if ".xml" in filename.lower():
          newscan.xmlpath = os.path.join(parent, filename)
        elif ".dcm" in filename.lower():
          fno = filename.split('.')
          tdf = df[df['seriesuid']==fno[0]]
          if not tdf.empty:
            print tdf
          fno = int(fno[0])
          data = dicom.read_file(os.path.join(parent, filename))
          if newscan.id == None:
            sno = parent.split('/')
            sno = sno[-1]
            newscan.id = sno
          #print data['0040','071A']
          newscan.sliceID[data['0008','0018'].value] = fno
          newscan.locationID[(float)(data['0020','1041'].value)] = str(os.path.join(parent, filename))
          locationList.append(data['0020','1041'].value)
          xyz = data['0020','0032'].value
          locationListAlt.append(xyz[2])
          spacing = data.PixelSpacing
          thickness = data.SliceThickness
      if thickness > 2.5:
        continue
      newscan.spacing = (spacing[0],spacing[0],thickness)
      count = 0
      locationList.sort()
      locationListAlt.sort()
      while locationList:
        i = locationList.pop()
        j = locationListAlt.pop()
        newscan.sliceList.append(newscan.locationID[i])
        newscan.locationDictAlt[i] = count
        newscan.locationDict[j] = count
        count += 1
      scans.append(newscan)
  return scans

In [108]:
def addCharacteristics(chars, nodule):
  try:
    subtlety = chars.find("{http://www.nih.gov}subtlety")
    nodule.subtlety = int(subtlety.text)
    internal = chars.find("{http://www.nih.gov}internalStructure")
    nodule.internalStructure = int(internal.text)
    calcification = chars.find("{http://www.nih.gov}calcification")
    nodule.calcification = int(calcification.text)
    sphericity = chars.find("{http://www.nih.gov}sphericity")
    nodule.sphericity = int(sphericity.text)
    margin = chars.find("{http://www.nih.gov}margin")
    nodule.margin = int(margin.text)
    lobulation = chars.find("{http://www.nih.gov}lobulation")
    nodule.lobulation = int(lobulation.text)
    spiculation = chars.find("{http://www.nih.gov}spiculation")
    nodule.spiculation = int(spiculation.text)
    texture = chars.find("{http://www.nih.gov}texture")
    nodule.texture = int(texture.text)
    ##-MALIGNANCY IS ASSUMING 60 YEAR OLD MALE SMOKER ..
    malignancy = chars.find("{http://www.nih.gov}malignancy")
    nodule.malignancy = int(malignancy.text)
    return nodule
  except AttributeError:
    nodule.AttributeError = True
    return nodule

In [117]:
def readXML(scan):
  print scan.id
  annotations = et.parse(scan.xmlpath)
  append = True
  header = annotations.find('./{http://www.nih.gov}ResponseHeader')
  seriesuid = header.find('./{http://www.nih.gov}SeriesInstanceUid')
  print seriesuid.text
  for reader in annotations.findall('./{http://www.nih.gov}readingSession'):
    for reading in reader.findall('./{http://www.nih.gov}unblindedReadNodule'):
      nn = Nodule() #new nodule
      if reading.find('./{http://www.nih.gov}characteristics') is not None:
        nn.type = 'L'
        characteristics = reading.find('./{http://www.nih.gov}characteristics')
        nn = addCharacteristics(characteristics, nn)
      else:
        nn.type = 'S'
      for roi in reading.findall('./{http://www.nih.gov}roi'):
        z = roi.find('./{http://www.nih.gov}imageZposition')
        try:
          plane = scan.locationDict[float(z.text)]
        except KeyError:
          print "KeyError XML"
          try:
            plane = scan.locationDictAlt[float(z.text)]
          except KeyError:
            append = False
            print "KeyError 2 XML"
            break
        nn.planes.append(plane)
        include = roi.find('./{http://www.nih.gov}inclusion')
        if nn.type == 'S':
          e = roi.find('./{http://www.nih.gov}edgeMap')
          x = e.find('./{http://www.nih.gov}xCoord')
          nn.ox = int(x.text)
          y = e.find('./{http://www.nih.gov}yCoord')
          nn.oy = int(y.text)
          nn.diameter = 3.
        elif nn.type == 'L':
          nn.roi[plane] = []
          if include.text == 'TRUE':
            for e in roi.findall('./{http://www.nih.gov}edgeMap'):
              x = e.find('./{http://www.nih.gov}xCoord')
              y = e.find('./{http://www.nih.gov}yCoord')
              coord = int(x.text),int(y.text)
              nn.roi[plane].append(coord)
      if nn.type=='L':
        nn.getProperties(scan.spacing)
      if append:
        scan.nodules.append(nn)
      append = True
    
    for reading in reader.findall("./{http://www.nih.gov}nonNodule"):
      nn = Nodule()
      nn.type = 'N'
      z = reading.find("./{http://www.nih.gov}imageZposition")
      try:
        plane = scan.locationDict[float(z.text)]
      except KeyError:
        print "KeyError XML"
        try:
          plane = scan.locationDictAlt[float(z.text)]
        except KeyError:
          append = False
          print "KeyError 2 XML"
      if append:
        nn.plane = plane
        locus = reading.find('./{http://www.nih.gov}locus')
        x = locus.find("./{http://www.nih.gov}xCoord")
        nn.ox = int(x.text)
        y = locus.find("./{http://www.nih.gov}yCoord")
        nn.oy = int(y.text)
        nn.diameter = 5.
        scan.nodules.append(nn)
  return scan

In [113]:
t1 = time.time()
scans = getFilesList(BIN3PATH)
t2 = time.time()
print str((t2-t1)/60) + " minutes"

5.62958888213 minutes


In [127]:
ct = 0
t1 = time.time()
for scan in scans:
  readXML(scan)
  ct += 1
  if ct%10==0:
    print ct
t2 = time.time()
print str((t2-t1)/60) + " minutes"

1.3.6.1.4.1.14519.5.2.1.6279.6001.160216916075817913953530562493


IOError: [Errno 2] No such file or directory: '/scr/nodules/lidc-idri/bin3/1.3.6.1.4.1.14519.5.2.1.6279.6001.160216916075817913953530562493/056.xml'

In [79]:
ct = 0
t1 = time.time()
for scan in scans:
  cluster(scan)
  ct += len(scan.largeGroups)
  if ct%10==0:
    print ct
t2 = time.time()
print str((t2-t1)/60) + " minutes"

1.3.6.1.4.1.14519.5.2.1.6279.6001.160216916075817913953530562493


TypeError: 'NoneType' object has no attribute '__getitem__'

In [64]:
scans[0].nodules[0].groupMembers

[1, 2, 4, 5, 6, 7, 10, 11, 12]

In [51]:
total = 0

In [52]:
#bin0 134
total += ct
print ct

134


In [59]:
#bin1 128
total += ct
print ct

128


In [63]:
#bin2 
total += ct
print ct

129


In [ ]:
#bin3
total += ct
print ct

In [ ]:
#bin4
total += ct
print ct

In [ ]:
#bin5
total += ct
print ct

In [ ]:
#bin6
total += ct
print ct

In [ ]:
#bin7
total += ct
print ct

In [ ]:
#bin8
total += ct
print ct

In [ ]:
#bin9
total += ct
print ct